In [6]:
import logging
from itertools import combinations, permutations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx
import random



from icecream import ic

logging.basicConfig(level=logging.DEBUG)

## Read the file and properly initialize data structures

In [7]:
CITIES = pd.read_csv('cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Ancona,43.60,13.50
1,Andria,41.23,16.29
2,Bari,41.12,16.87
3,Bergamo,45.70,9.67
4,Bologna,44.50,11.34


## Cost and fitness functions

In [8]:
def tsp_cost(tsp):
    assert tsp[0] == tsp[-1]
    cluster_set = set(tsp[:-1])  # Escludi l'ultima città (uguale alla prima)
    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost if cluster_set.issubset(set(range(len(CITIES)))) else float('inf')

def fitness(solution):
    return -tsp_cost(solution)

## Fast solution  
The first point required by the lab text was to provide an algorithm able to produce a solution in a 'fast' way. In order to fulfill the aim, I've used a greedy algorithm and then, to the valid tour provided by the gredy, I simply apply simulated annealing. On my pc, even on larger instances, it is able to provide a solution even in less than 1 second (I know that time is not a unit measure, but, since the fitness is not so clear and well defined in this context, at least there's a reference of execution time on a machine).  
The greedy algorithm simply consists on starting from a random city and constructing the path step by step by picking the closest city to the current one. Parameters that have been set for the simulated annealing have been tuned based on empirical trials done and better explained in the file trials_tsp.ipynb, available in this folder.

In [9]:
''' 
The algorithm takes as argumenti the integer representing the starting city in the path and return a list
of integers representing the path.
'''
def greedy(startingcity: int):
    visited = np.full(len(CITIES), False)
    dist = DIST_MATRIX.copy()
    city = startingcity
    visited[city] = True
    tsp = list()
    tsp.append(int(city))
    while not np.all(visited):
        dist[:, city] = np.inf
        closest = np.argmin(dist[city])
        visited[closest] = True
        city = closest
        tsp.append(int(city))
    tsp.append(tsp[0])
    return tsp

#Choose a random city and execute the algorithm
city = np.random.randint(len(CITIES))
tsp = greedy(city)

logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {tsp_cost(tsp):.2f}km")

INFO:root:result: Found a path of 46 steps, total length 5136.22km


In [10]:
# Simulated annealing parameters 
initial_temp = 0.1  # Starting temperature
cooling_rate = 0.999  # How to slow down the cooling
min_temp = 1e-5  # Minimum temperature 

def two_opt(tour):
    """2-opt to invert a portion of the tour"""
    i, j = np.sort(np.random.randint(1, len(CITIES) - 1, size=2))
    new_tour = tour[:i] + tour[i:j+1][::-1] + tour[j+1:]
    return new_tour

def simulated_annealing(tsp, initial_temp, cooling_rate, min_temp):
    current_solution = tsp.copy()
    current_cost = tsp_cost(current_solution)
    best_solution = current_solution.copy()
    best_cost = current_cost
    temperature = initial_temp

    while temperature > min_temp:
        # Generate a new solution through 2-opt
        new_solution = two_opt(current_solution)

        # Cost of the new solution
        new_cost = tsp_cost(new_solution)

        # Cost difference
        cost_diff = new_cost - current_cost

        # Decide whether to accept the new solution
        if cost_diff < 0 or np.random.rand() < np.exp(-cost_diff / temperature):
            current_solution = new_solution
            current_cost = new_cost
            if current_cost < best_cost:
                best_solution = current_solution
                best_cost = current_cost

        # Cool down the temperature
        temperature *= cooling_rate

    return best_solution, best_cost

best_solution, best_cost = simulated_annealing(tsp, initial_temp, cooling_rate, min_temp)

logging.info(f"Simulated annealing optimized solution cost: {best_cost:.2f} km")

INFO:root:Simulated annealing optimized solution cost: 4704.68 km


## Second solution, more time consuming but more accurate and efficient

The second algorithm provided for this lab is a mix of many strategies. The skeleton is given by the genetuic algorithm (steady state model) that uses, as a flow, the modern strategy (pick two parents, perform crossover and mutate the child before adding it to the offspring). Regarding mutation and crossover, I decided to use 2-opt, as I did for simulated annealing, for mutation, while I used the inver Over crossover. In reality, the inver Over crossover have been slightly modified in order to execute, with a certain (low) probability simulated annealing on the child produced by the two parents. This strategy allowed to reach much better results. As I did for the first algorithm, also in this case parameters have been fine-tuned in order to obtain better results and other considerations are left to trials_tsp.ipynb where you can see the entire flow of reasoning, if interested.
As starting point for the EA many trials have been done, at the end I decided to use both efficient solutions and randomic solutions. More efficient solutions are added to the initial population using the first algorithm (the fast one). You can have a look at their percentage in the code below, or even to results obtained by changing them in the aforementioned file.

In [11]:
# Function to create a random solution (valid, it is a tour of the cities)
def create_random_solution():
    solution = list(range(len(CITIES)))
    np.random.shuffle(solution)
    solution.append(solution[0])  
    return solution

'''
This function receives total_size (population_size), greedy_size (number of individuals generated using the greedy algorithm), 
random_size (number of individuals randomly generated) and simulated_annealing_size (number of individuals generated 
using greedy optimized using the simulated annealing algorithm with 2-opt mutation).
'''
def create_initial_population(total_size,greedy_size,random_size,simulated_annealing_size):
    population = []
    for _ in range(greedy_size):
        population.append(greedy(np.random.randint(0,len(CITIES))))
    for _ in range(random_size):
        population.append(create_random_solution())
    for _ in range(simulated_annealing_size):
        population.append(simulated_annealing(greedy(np.random.randint(0,len(CITIES))),initial_temp,cooling_rate,min_temp)[0])
    for _ in range(total_size-len(population)):
        population.append(create_random_solution())
    return population


# Tournament selection
def tournament_selection(population, k=30):
    selected = random.sample(population, k)
    selected.sort(key=fitness, reverse=True)
    return selected[0]

def inversion_crossover(parent1, parent2):
    # Determine the size of the parents (excluding the last element)
    size = len(parent1) - 1  
    # Randomly choose two crossover points
    start, end = sorted(random.sample(range(size), 2))

    # Copy the segment from parent1 and invert it
    child_segment = parent1[start:end + 1][::-1]  # Invert the selected segment
    
    # Create a child list that initially includes None
    child = [None] * size
    
    # Fill in the inverted segment into the child
    child[start:end + 1] = child_segment
    
    # Fill the remaining positions with genes from parent2, avoiding duplicates
    p2_index = 0
    for i in range(size):
        if child[i] is None:
            # Find the next gene from parent2 that isn't already in the child
            while parent2[p2_index] in child:
                p2_index += 1
            child[i] = parent2[p2_index]

    child.append(child[0])  # Close the cycle by adding the first city again
    if(random.random()<0.05): #first tests were done with 0.01
        child,_=simulated_annealing(child,initial_temp,cooling_rate,min_temp)
    return child

#2-opt
def mutate(solution, mutation_rate):
    if random.random() < mutation_rate:
        i, j = sorted(random.sample(range(1, len(solution) - 1), 2))
        solution[i:j] = reversed(solution[i:j])
    return solution

In [12]:
# Algorithm parameters
POPULATION_SIZE = 150 
GENERATIONS = 100 
ELITE_SIZE = 20  
MUTATION_RATE = 0.5

# EA -> Modern GA
def evolutionary_algorithm():
    # Initial population setup
    population = create_initial_population(
        POPULATION_SIZE, int(POPULATION_SIZE * 0), int(0.95 * POPULATION_SIZE), int(0.05 * POPULATION_SIZE)
    )
    best_solution = None
    best_fitness = float('-inf')

    for generation in range(GENERATIONS):
        # Sort the population by fitness in descending order
        population.sort(key=fitness, reverse=True)
        
        # Select the best individual as elite
        elite = population[:ELITE_SIZE]  # Only keep the top individual
        
        # Update the best individual if needed
        if fitness(elite[0]) > best_fitness:
            best_solution = elite[0]
            best_fitness = fitness(best_solution)

        # New population setup
        new_population = elite[:]  # Keep only the best individual
        while len(new_population) < POPULATION_SIZE:
            # Selection and crossover
            parent1 = tournament_selection(population)
            parent2 = tournament_selection(population)
            child = inversion_crossover(parent1, parent2)
            child = mutate(child, MUTATION_RATE)
            new_population.append(child)
        
        # Update population for the next generation
        population = new_population

        # Display progress every 20 generations
        if generation % 20 == 0:
            print(f"Generation {generation}: Best tour length = {-best_fitness:.2f} km")

    return best_solution, -best_fitness

# Run the evolutionary algorithm
best_tour, best_cost = evolutionary_algorithm()

print(f"Best tour found: {best_tour} with total length {best_cost:.2f} km")

Generation 0: Best tour length = 4315.21 km
Generation 20: Best tour length = 4181.62 km
Generation 40: Best tour length = 4172.76 km
Generation 60: Best tour length = 4172.76 km
Generation 80: Best tour length = 4172.76 km
Best tour found: [18, 20, 3, 6, 44, 45, 40, 5, 41, 43, 23, 9, 30, 12, 33, 0, 27, 26, 39, 34, 15, 14, 21, 35, 11, 1, 2, 38, 17, 31, 37, 8, 24, 7, 36, 16, 10, 29, 4, 19, 32, 25, 28, 13, 42, 22, 18] with total length 4172.76 km
